# Data Transformation

In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

%config IPCompleter.greedy=True

import warnings
warnings.simplefilter(action='ignore')

In [50]:
cat = pd.read_pickle('../../02_Data/03_Work/cat_result_eda.pickle')
num = pd.read_pickle('../../02_Data/03_Work/num_result_eda.pickle')


## New Variables

In [52]:
df = pd.concat([cat,num], axis=1)
df

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,sales,sell_price
date,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,1.25
2013-01-01,CA_3,FOODS_3_120,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,4.98
2013-01-01,CA_3,FOODS_3_202,d_704,2013,1,4,Tuesday,NewYear,National,11249,20,4.28
2013-01-01,CA_3,FOODS_3_252,d_704,2013,1,4,Tuesday,NewYear,National,11249,34,1.48
2013-01-01,CA_3,FOODS_3_288,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,CA_4,FOODS_3_329,d_1767,2015,11,3,Monday,No_event,No_event,11544,5,2.98
2015-11-30,CA_4,FOODS_3_555,d_1767,2015,11,3,Monday,No_event,No_event,11544,4,1.68
2015-11-30,CA_4,FOODS_3_586,d_1767,2015,11,3,Monday,No_event,No_event,11544,9,1.68


We already have 
- date components
- calendar variables

We need:
- the ones identified at EDA
- lags 
- rolling windows

### Intermitent demand variable

In [8]:
# other related variable: low/high sells period

Will identify how many consecutive days have passed with 0 sales.

If n days at 0 sales, then it is out of stock.
We can make different ones using different n


In [53]:
def out_of_stock(sales, n = 5):
    zero_sales = pd.Series(np.where(sales == 0,1,0))
    num_zeros = zero_sales.rolling(n).sum()
    out_of_stock = np.where(num_zeros == n,1,0)
    return(out_of_stock)

In [55]:
df = df.sort_values(by = ['store_id','item_id','date'])

In [56]:

df['out_of_stock_3'] = df.groupby(['store_id','item_id']).sales.transform(lambda x: out_of_stock(x, 3))
df['out_of_stock_7'] = df.groupby(['store_id','item_id']).sales.transform(lambda x: out_of_stock(x, 7))
df['out_of_stock_15']= df.groupby(['store_id','item_id']).sales.transform(lambda x: out_of_stock(x, 15))



### Lag Variables

In [57]:
# lags for the variables:

#      - sales: 15 days
#      - sell_price: 7 days
#      - out_of_stock: 1 day

In [58]:
def create_lags(df, variable, num_lags = 7):
    # create object dataframe
    lags = pd.DataFrame()
    
    # create lags
    for each in range(1,num_lags+1):
        lags[variable + '_lag_'+str(each)] = df[variable].shift(each)
        
    #return lags dataframe
    return lags


In [96]:
# sell price ---> 7 days lag
lags_sell_price_df = (df.groupby(['store_id', 'item_id'])
                        .apply(lambda x: create_lags(df = x, variable = 'sell_price', num_lags= 7), include_groups=False)
                        .reset_index()
                        .set_index('date'))

# out_of_stock ---> 1 day lag
lags_out_of_stock_3_df = (df.groupby(['store_id','item_id'])
                            .apply(lambda x: create_lags(df = x, variable = 'out_of_stock_3', num_lags= 1), include_groups=False)
                            .reset_index()
                            .set_index('date'))
lags_out_of_stock_7_df = (df.groupby(['store_id','item_id'])
                            .apply(lambda x: create_lags(df = x, variable = 'out_of_stock_7', num_lags= 1), include_groups=False)
                            .reset_index()
                            .set_index('date'))
lags_out_of_stock_15_df = (df.groupby(['store_id','item_id'])
                            .apply(lambda x: create_lags(df = x, variable = 'out_of_stock_15', num_lags= 1), include_groups=False)
                            .reset_index()
                            .set_index('date'))

# sales ----> 15 days lag
lags_sales_df = (df.groupby(['store_id','item_id'])
                    .apply(lambda x: create_lags(df = x, variable = 'sales', num_lags= 15))
                    .reset_index()
                    .set_index('date'))



### Rolling windows

In [61]:
lags_sales_df

,store_id,item_id,sales_lag_1,sales_lag_2,sales_lag_3,sales_lag_4,sales_lag_5,sales_lag_6,sales_lag_7,sales_lag_8,sales_lag_9,sales_lag_10,sales_lag_11,sales_lag_12,sales_lag_13,sales_lag_14,sales_lag_15
date,,,,,,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,CA_3,FOODS_3_090,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,CA_3,FOODS_3_090,224.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,CA_3,FOODS_3_090,241.0,224.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-05,CA_3,FOODS_3_090,232.0,241.0,224.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-26,CA_4,FOODS_3_714,6.0,12.0,3.0,15.0,13.0,15.0,5.0,6.0,12.0,8.0,17.0,17.0,13.0,10.0,7.0
2015-11-27,CA_4,FOODS_3_714,22.0,6.0,12.0,3.0,15.0,13.0,15.0,5.0,6.0,12.0,8.0,17.0,17.0,13.0,10.0
2015-11-28,CA_4,FOODS_3_714,10.0,22.0,6.0,12.0,3.0,15.0,13.0,15.0,5.0,6.0,12.0,8.0,17.0,17.0,13.0


In [39]:
# we are going to create 3 types:
### - local minimum
### - local mean 
### - local maximum

In [62]:
def local_min(df, variable, num_periods = 7):
    lmin = pd.DataFrame()
    
    for each in range(2,num_periods+1):
        lmin[variable+'_minlocal_'+str(each)] = df[variable].shift(1).rolling(each).min()
    
    return lmin
        
def local_max(df, variable, num_periods = 7):
    lmax = pd.DataFrame()
    
    for each in range(2, num_periods+1):
        lmax[variable+'_maxlocal_'+str(each)] = df[variable].shift(1).rolling(each).max()

    return lmax
        
def local_mean(df, variable, num_periods = 7):
    lmean = pd.DataFrame()
    
    for each in range(2,num_periods+1):
        lmean[variable+'_meanlocal_'+str(each)] = df[variable].shift(1).rolling(each).mean()
        
    return lmean
        

    


In [97]:
min_local_df = (df.groupby(['store_id','item_id'])
                  .apply(lambda x: local_min(df = x, variable = 'sales', num_periods= 15))
                  .reset_index()
                  .set_index('date'))
mean_local_df = (df.groupby(['store_id','item_id'])
                    .apply(lambda x: local_mean(df = x, variable = 'sales', num_periods= 15))
                    .reset_index()
                    .set_index('date'))
max_local_df = (df.groupby(['store_id','item_id'])
                    .apply(lambda x: local_max(df = x, variable = 'sales', num_periods= 15))
                    .reset_index()
                    .set_index('date'))


## Prepare datasets

In [67]:
df_joined = pd.concat([df,
                      lags_sell_price_df,
                      lags_out_of_stock_3_df,
                      lags_out_of_stock_7_df,
                      lags_out_of_stock_15_df,
                      lags_sales_df,
                      min_local_df,
                      mean_local_df,
                      max_local_df], axis = 1)

# delete duplicated columns
df_joined = df_joined.loc[:,~df_joined.columns.duplicated()]
df_joined

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,...,sales_maxlocal_6,sales_maxlocal_7,sales_maxlocal_8,sales_maxlocal_9,sales_maxlocal_10,sales_maxlocal_11,sales_maxlocal_12,sales_maxlocal_13,sales_maxlocal_14,sales_maxlocal_15
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,CA_3,FOODS_3_090,d_705,2013,1,5,Wednesday,No_event,No_event,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,CA_3,FOODS_3_090,d_706,2013,1,6,Thursday,No_event,No_event,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,CA_3,FOODS_3_090,d_707,2013,1,7,Friday,No_event,No_event,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-05,CA_3,FOODS_3_090,d_708,2013,1,1,Saturday,No_event,No_event,11250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-26,CA_4,FOODS_3_714,d_1763,2015,11,6,Thursday,Thanksgiving,National,11543,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
2015-11-27,CA_4,FOODS_3_714,d_1764,2015,11,7,Friday,No_event,No_event,11543,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
2015-11-28,CA_4,FOODS_3_714,d_1765,2015,11,1,Saturday,No_event,No_event,11544,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


In [69]:
df_joined.dropna(inplace=True)

### Delete the variables we are not going to use to model

In [75]:
# since we are doing forecasting, 
# we cannot use the variables we will not know in advance
to_delete = ['d','wm_yr_wk','sell_price','out_of_stock_3','out_of_stock_7','out_of_stock_15']


In [76]:
df_joined.drop(columns=to_delete, inplace=True)

In [77]:
# identify target
target = df_joined.sales

In [78]:
# separe num and cat

cat = df_joined.select_dtypes(include = 'O')
num = df_joined.select_dtypes(exclude = 'O')


## Categoric Transformation

### One hot encoding

In [79]:
var_ohe = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

In [81]:

ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')


In [83]:
cat_ohe = ohe.fit_transform(cat[var_ohe])
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())



### Target Encoding

In [84]:
var_te = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

In [89]:
te = TargetEncoder(min_samples_leaf=100, return_df = False)


In [98]:
cat_te = te.fit_transform(cat[var_te], y = target)


In [99]:

names_te = [variable + '_te' for variable in var_te]
cat_te = pd.DataFrame(cat_te, columns = names_te)


## Unify transformed dataframes

In [105]:
from_df_joined = df_joined[['store_id','item_id']].reset_index()

from_df_joined.head(2)

,date,store_id,item_id
0,2013-01-16,CA_3,FOODS_3_090
1,2013-01-17,CA_3,FOODS_3_090


In [106]:

dataframes = [from_df_joined, cat_ohe, cat_te, num.reset_index(drop=True)]


In [107]:
df_board = pd.concat(dataframes, axis = 1)

df_board

,date,store_id,item_id,year_2013,year_2014,year_2015,month_1,month_2,month_3,month_4,...,sales_maxlocal_6,sales_maxlocal_7,sales_maxlocal_8,sales_maxlocal_9,sales_maxlocal_10,sales_maxlocal_11,sales_maxlocal_12,sales_maxlocal_13,sales_maxlocal_14,sales_maxlocal_15
0,2013-01-16,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
1,2013-01-17,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
2,2013-01-18,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
3,2013-01-19,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,281.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
4,2013-01-20,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20975,2015-11-26,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
20976,2015-11-27,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20977,2015-11-28,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20978,2015-11-29,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


### Save the dataset

In [108]:

df_board.to_pickle('../../02_Data/03_Work/df_board.pickle')
